#### **Imports**

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

#### **Date Gathering and Preprocessig**

##### *Dowloading*

In [2]:
# This is for downloading Apple stock data
data = yf.download(tickers = ['AAPL'], start="1980-12-12", end="2025-04-01")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
data.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
1980-12-12,0.098726,0.099155,0.098726,0.098726,469033600
1980-12-15,0.093575,0.094005,0.093575,0.094005,175884800
1980-12-16,0.086707,0.087136,0.086707,0.087136,105728000
1980-12-17,0.088853,0.089282,0.088853,0.088853,86441600
1980-12-18,0.091429,0.091858,0.091429,0.091429,73449600


##### *Formating and storing*

In [4]:
# Storing date as a column instead of index
data = data.reset_index()

In [5]:
data

Price,Date,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
0,1980-12-12,0.098726,0.099155,0.098726,0.098726,469033600
1,1980-12-15,0.093575,0.094005,0.093575,0.094005,175884800
2,1980-12-16,0.086707,0.087136,0.086707,0.087136,105728000
3,1980-12-17,0.088853,0.089282,0.088853,0.088853,86441600
4,1980-12-18,0.091429,0.091858,0.091429,0.091429,73449600
...,...,...,...,...,...,...
11160,2025-03-25,223.750000,224.100006,220.080002,220.770004,34493600
11161,2025-03-26,221.529999,225.020004,220.470001,223.509995,34466100
11162,2025-03-27,223.850006,224.990005,220.559998,221.389999,37094800


In [6]:
# Removing multilevel index
df = data.droplevel(level=0,axis=1)

In [7]:
# Assigning column names for further use
cols = ['date','close','high','low','open','volume']
df.columns = cols

In [8]:
df.head()

,date,close,high,low,open,volume
0,1980-12-12,0.098726,0.099155,0.098726,0.098726,469033600
1,1980-12-15,0.093575,0.094005,0.093575,0.094005,175884800
2,1980-12-16,0.086707,0.087136,0.086707,0.087136,105728000
3,1980-12-17,0.088853,0.089282,0.088853,0.088853,86441600
4,1980-12-18,0.091429,0.091858,0.091429,0.091429,73449600


In [9]:
# Saving to a CSV file
df.to_csv("../data/raw/Stocks1.csv", index = False)

##### *Basic Analysis*

In [10]:
# Check for column names and categoris
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11165 entries, 0 to 11164
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    11165 non-null  datetime64[ns]
 1   close   11165 non-null  float64       
 2   high    11165 non-null  float64       
 3   low     11165 non-null  float64       
 4   open    11165 non-null  float64       
 5   volume  11165 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 523.5 KB


In [11]:
# Shape of dataset
df.shape

(11165, 6)

In [12]:
# Check for duplicates
df.duplicated().sum()

np.int64(0)

In [13]:
# Statistics of the date
df.describe()

,date,close,high,low,open,volume
count,11165,11165.000000,11165.000000,11165.000000,11165.000000,1.116500e+04
mean,2003-01-25 17:29:12.440662784,24.575112,24.819920,24.305480,24.555977,3.140738e+08
min,1980-12-12 00:00:00,0.037773,0.038203,0.037773,0.038203,0.000000e+00
25%,1991-12-27 00:00:00,0.244305,0.249395,0.238868,0.244269,1.099380e+08
50%,2003-01-23 00:00:00,0.453388,0.460808,0.447155,0.453360,2.024736e+08
75%,2014-02-25 00:00:00,18.787733,18.936100,18.610886,18.788937,3.945144e+08
max,2025-03-31 00:00:00,258.735504,259.814335,257.347047,257.906429,7.421641e+09
std,NaN,51.787642,52.288254,51.225759,51.736698,3.345346e+08


#### **Feature Construction**

##### *Adjusted close*

**Concept of Splits**
* Normally for each share bought the price is retured for invested in that share
* But sometimes the share gets split i.e the same amount invested is used in multiple shares
* The returns of this share is given individually but that should be the sum of all shares

**Example :** On day 1 if a share is bought for 100 the close price will be 100. On day 2 if 2-for-1 split has happened, the share is split into 50 and 50, but the close price will be shown as 50 only but it should be 50 * 2 = 100.

**Issue :** When we use model like LSTM, the sudden fall in price due to split may cause the model to fail


In [14]:
# Get the dates and split ratio
stock = yf.Ticker('AAPL')
splits = stock.splits
splits = splits.reset_index()
splits.columns = ['date', 'split_ratio']
splits

,date,split_ratio
0,1987-06-16 00:00:00-04:00,2.0
1,2000-06-21 00:00:00-04:00,2.0
2,2005-02-28 00:00:00-05:00,2.0
3,2014-06-09 00:00:00-04:00,7.0
4,2020-08-31 00:00:00-04:00,4.0


**Split_ratio** : previous close / current close

In [15]:
splits.date.dtype

datetime64[ns, America/New_York]

In [16]:
# convert date type from datetime64[ns, America/New_York] to datetime64[ns]
splits['date'] = splits["date"].dt.tz_localize(None)

In [17]:
# add split ratio column to original data
df = df.merge(splits, on='date', how='left')

In [18]:
# replace null values with 1 showing no split took place
df['split_ratio'].fillna(1, inplace=True)

C:\Users\smoha\AppData\Local\Temp\ipykernel_8060\2285346929.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['split_ratio'].fillna(1, inplace=True)


- **Adj_factor** : It is simply the number of dividends that happened. The cummulative product of split ratio gives how many times and how many dividends occurs until that date
- **Example** : If on day 2 it is 2-for-1 and on day 3 it is 2-for-1, then it becomes 2 * 2 on day 4 (as the 2 dividends of day2 are again divided into 2 parts each on day 3)

In [19]:
df['adj_factor'] = df['split_ratio'].cumprod()

In [20]:
# the product of no. of dividends and close price gives adj_close
df['adj_close'] = df['adj_factor'] * df['close']

##### *Moving Averages*

**Moving Average :** The averge of a certain window helps in analysing the adj close value of that terms better. It is simple mean of n windows.

In [21]:
# df['adj_close'].rolling(window=10).mean() # standard moving average
# exponential moving average
df['rolling_ma'] = df['adj_close'].ewm(span=10, adjust=False).mean()

##### *RSI*

**RSI (Relative Strength Index)** is a momentum indicator used in stock trading to measure the speed and magnitude of price changes. It helps traders identify overbought or oversold conditions.

RSI values range from 0 to 100:

- Above 70 : Overbought (price may fall soon)
- Below 30 : Oversold (price may rise soon)
- Between 30-70 : Neutral (no strong trend)


- **Relative Strenght** = *(Average gain for N periods / Avg loss for N periods)*
- **RSI** = *100 - (100 / 1 - RS)*

In [22]:
# curr close value - past close value
df['change'] = df['adj_close'].diff()

In [23]:
# take gain if gain, else take 0
df['gain'] = np.where(df['change'] > 0, df['change'], 0)
# take loss if loss, else take 0
df['loss'] = np.where(df['change'] < 0, -df['change'], 0)

In [24]:
# calculate average gain and loss usign exponentially moving average
df['avg_gain'] = df['gain'].ewm(span=14, adjust=False).mean()
df['avg_loss'] = df['loss'].ewm(span=14, adjust=False).mean()

In [25]:
df.loc[:,['date','adj_close','change','gain','loss','avg_gain','avg_loss']].head()

,date,adj_close,change,gain,loss,avg_gain,avg_loss
0,1980-12-12,0.098726,NaN,0.000000,0.000000,0.000000,0.000000
1,1980-12-15,0.093575,-0.005151,0.000000,0.005151,0.000000,0.000687
2,1980-12-16,0.086707,-0.006868,0.000000,0.006868,0.000000,0.001511
3,1980-12-17,0.088853,0.002146,0.002146,0.000000,0.000286,0.001309
4,1980-12-18,0.091429,0.002576,0.002576,0.000000,0.000591,0.001135


In [26]:
df['relative_strength'] = df['avg_gain'] / df['avg_loss']

In [27]:
df['RSI'] = 100 - (100 / (1 + df["relative_strength"]))

##### *MACD*

**MACD** is a momentum indicator that helps identify trend direction, strength, and potential reversals in stock prices. It is calculated using two Exponential Moving Averages (EMAs).

*MACD Calculation Formula*
- **MACD Line** = 12-day EMA – 26-day EMA
- **Signal Line** = 9-day EMA of the MACD Line
- **MACD Histogram** = MACD Line – Signal Line

- If MACD > Signal Line → Bullish (buy signal)
- If MACD < Signal Line → Bearish (sell signal)



In [28]:
df["EMA_12"] = df["adj_close"].ewm(span=12, adjust=False).mean()
df["EMA_26"] = df["adj_close"].ewm(span=26, adjust=False).mean()

# Calculate MACD Line
df["MACD"] = df["EMA_12"] - df["EMA_26"]

# Calculate Signal Line (9-day EMA of MACD)
df["Signal_Line"] = df["MACD"].ewm(span=9, adjust=False).mean()

# Calculate MACD Histogram
df["MACD_Histogram"] = df["MACD"] - df["Signal_Line"]

In [29]:
df.loc[:,['EMA_12','EMA_26','MACD','Signal_Line','MACD_Histogram']].head()

,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram
0,0.098726,0.098726,0.000000,0.000000,0.000000
1,0.097934,0.098344,-0.000411,-0.000082,-0.000329
2,0.096206,0.097482,-0.001276,-0.000321,-0.000955
3,0.095075,0.096843,-0.001768,-0.000610,-0.001158
4,0.094514,0.096442,-0.001928,-0.000874,-0.001054


##### *Bollinger brand*

**Bollinger Bands** are a volatility indicator that helps identify overbought and oversold conditions in a stock. They consist of three lines:

- *Upper Band* = Moving Average + (2 × Standard Deviation)
- *Middle Band* = Simple Moving Average (SMA) (default: 20-day)
- *Lower Band* = Moving Average - (2 × Standard Deviation)

In [30]:
df["SMA_20"] = df["adj_close"].ewm(span=20, adjust=False).mean()

# Standard Deviation
df["STD"] = df["adj_close"].ewm(span=20, adjust=False).std()

# Calculate Bollinger Bands
df["upper_band"] = df["SMA_20"] + (2 * df["STD"])
df["lower_band"] = df["SMA_20"] - (2 * df["STD"])

In [31]:
df.loc[:, ['SMA_20', 'STD', 'upper_band', 'lower_band']].head()

,SMA_20,STD,upper_band,lower_band
0,0.098726,NaN,NaN,NaN
1,0.098235,0.003642,0.105519,0.090951
2,0.097137,0.006568,0.110273,0.084001
3,0.096348,0.006505,0.109358,0.083339
4,0.095880,0.005946,0.107771,0.083989


##### *Date*

Splitting of date may help you find the seasonality and different trends in the data

In [32]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday

##### *Adding Seasonality*
- Sin, cos terms repeat themselves periodically
- Her fourier_sin_7, fourier_Cos_7 represents the weekly repetition (seasonality)
- Her fourier_sin_30, fourier_Cos_30 represents the monthly repetition (seasonality)

In [33]:

df["Fourier_Sin_7"] = np.sin(2 * np.pi * df.date.dt.day_of_year / 7)
df["Fourier_Cos_7"] = np.cos(2 * np.pi * df.date.dt.day_of_year / 7)
df["Fourier_Sin_30"] = np.sin(2 * np.pi * df.date.dt.day_of_year / 30)
df["Fourier_Cos_30"] = np.cos(2 * np.pi * df.date.dt.day_of_year / 30)


##### *Saving the feature constructed data*

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11165 entries, 0 to 11164
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               11165 non-null  datetime64[ns]
 1   close              11165 non-null  float64       
 2   high               11165 non-null  float64       
 3   low                11165 non-null  float64       
 4   open               11165 non-null  float64       
 5   volume             11165 non-null  int64         
 6   split_ratio        11165 non-null  float64       
 7   adj_factor         11165 non-null  float64       
 8   adj_close          11165 non-null  float64       
 9   rolling_ma         11165 non-null  float64       
 10  change             11164 non-null  float64       
 11  gain               11165 non-null  float64       
 12  loss               11165 non-null  float64       
 13  avg_gain           11165 non-null  float64       
 14  avg_lo

In [35]:
df.dropna(inplace=True)

In [36]:
df.head()

,date,close,high,low,open,volume,split_ratio,adj_factor,adj_close,rolling_ma,...,upper_band,lower_band,year,month,day,weekday,Fourier_Sin_7,Fourier_Cos_7,Fourier_Sin_30,Fourier_Cos_30
1,1980-12-15,0.093575,0.094005,0.093575,0.094005,175884800,1.0,1.0,0.093575,0.097789,...,0.105519,0.090951,1980,12,15,0,-5.487903e-14,1.000000,-0.866025,-0.500000
2,1980-12-16,0.086707,0.087136,0.086707,0.087136,105728000,1.0,1.0,0.086707,0.095774,...,0.110273,0.084001,1980,12,16,1,7.818315e-01,0.623490,-0.951057,-0.309017
3,1980-12-17,0.088853,0.089282,0.088853,0.088853,86441600,1.0,1.0,0.088853,0.094516,...,0.109358,0.083339,1980,12,17,2,9.749279e-01,-0.222521,-0.994522,-0.104528
4,1980-12-18,0.091429,0.091858,0.091429,0.091429,73449600,1.0,1.0,0.091429,0.093955,...,0.107771,0.083989,1980,12,18,3,4.338837e-01,-0.900969,-0.994522,0.104528
5,1980-12-19,0.097009,0.097438,0.097009,0.097009,48630400,1.0,1.0,0.097009,0.094510,...,0.106579,0.085396,1980,12,19,4,-4.338837e-01,-0.900969,-0.951057,0.309017


In [37]:
df.to_csv('../data/interim/Stocks2.csv', index = False)